# Landscape of Vibrational Anharmonicity Scores for Cubic Perovskites

This notebook provides you the code to analyze the vibrational anharmonicity scores of cubic perovskites at $T=$300 K with respect to either their 'convex-hull' stabilities $\Delta H_c$, or their Goldschmidt tolerace factors. Practically, as shown in Notebook #1, the time-dependent vibrational anharmonicity scores $\sigma(T)$ is calculated for every frame of the *ab initio* MD trajectory [to be denoted as $\sigma(T,t)$]. Here, the $\sigma(T)$ values chartted for each cubic perovskites will be the $\sigma(T,t)$ values ***averaged across the entire MD trajectory***.

In [8]:
from tolerance_factor import tolerance_factor
from chemical_space import *
from ase.db import connect

In [9]:
X='O'

#get the corresponding A, B cations that go with the anion
if X in halide_C:
    A = halide_A
    B = halide_B
elif X in chalco_C:
    A = chalco_A
    B = chalco_B

#connect to the corresponding database toto retrieve the data
db=connect('./perovskites_'+str(X)+'.db')

In [10]:
tolerance_factors=[]
energy_differences=[] #as the formation energies of Pm3m perovskite to the most stable random structure
label=[]
sigmas=[]

for a in A:
    for b in B:
        row = None
        system_name = a + b + X
        uid = system_name + '_Pm3m'
        pm3m_formation_e = None
        
        tolerance_f = tolerance_factor(a, b, X)
        
        #Get the database entry corresponding to this cubic perovskite
        try:
            row = db.get(selection=[('uid', '=', uid)])
        except:
            continue
            
        if row is not None:
            #formation energy for the cubic perovskite
            try:
                pm3m_formation_e = row.key_value_pairs['formation_energy']
            except KeyError:
                continue
                
            sigma = None
            try:
                sigma = row.key_value_pairs['sigma_300K_single']
            except:
                pass
                
            if pm3m_formation_e is not None:
                #Get the formation energies for the most stable structure that is optimized from a random starting 
                #point. In the actual computational screening procedure, 10 structures were screened for each compound.
                #However, due to the storage limit of the Github repository, only the information on the 
                #lowest energy structure is stored. This bit of the code is trying to digging that information out
                random_energy = None
                counter = 0
                
                while (random_energy is None) or (counter<10):
                    uid_r = uid + '_rand_str_' + str(counter)
                    try:
                        row = db.get(selection=[('uid', '=', uid_r)])
                    except:
                        pass
                    
                    if row is not None:
                        random_energy = row.key_value_pairs['formation_energy']
                    counter+=1

                if (random_energy is not None) and (sigma is not None):
                    energy_differences.append(pm3m_formation_e - random_energy)
                    tolerance_factors.append(tolerance_f)
                    sigmas.append(sigma)
                    label.append(system_name+'3')

print("Total number of available data: "+str(len(energy_differences)))

Total number of available data: 498


In [11]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Div
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.layouts import row

output_notebook()

data = {'x1':tolerance_factors,
        'x2':energy_differences,
        'y':sigmas,
        'label':label}
source = ColumnDataSource(data=data)

p1 = figure(width=600, height=600, background_fill_color="#fafafa")
p1.circle(x='x1', y='y', source=source,size=10,color='#CB0000',alpha=0.6)
hover = HoverTool()
hover.tooltips=[
    ('t', '@x1'),
    ('sigma', '@y'),
    ('Compound', '@label')
]
p1.add_tools(hover)

p1.xaxis.axis_label='Tolerance Factor'
p1.yaxis.axis_label="Sigma (300 K)"
p1.xaxis.axis_label_text_font_size = "20pt"
p1.yaxis.axis_label_text_font_size = "16pt"
p1.xaxis.major_label_text_font_size = "15pt"
p1.yaxis.major_label_text_font_size = "15pt"

#==========================================================

p2 = figure(width=600, height=600, background_fill_color="#fafafa")
p2.circle(x='x2', y='y', source=source,size=10,color='#FFBB00',alpha=0.6)
hover = HoverTool()
hover.tooltips=[
    ('Delta Hc', '@x2'),
    ('sigma', '@y'),
    ('Compound', '@label')
]
p2.add_tools(hover)

p2.xaxis.axis_label='Convex Hull Energies (eV/atom)'
p2.yaxis.axis_label="Sigma (300 K)"
p2.xaxis.axis_label_text_font_size = "20pt"
p2.yaxis.axis_label_text_font_size = "16pt"
p2.xaxis.major_label_text_font_size = "15pt"
p2.yaxis.major_label_text_font_size = "15pt"

show(row(p1,p2))

Loading BokehJS ...

In [12]:
#matrix plot ... to be implemented